In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import os, shutil, zipfile 
# 欲解壓縮的zip檔名 
data = ['train', 'test']
path = '../input/dogs-vs-cats-redux-kernels-edition/'
# 在當前的目錄解開train.zip、test.zip
for el in data:
    with zipfile.ZipFile(path + el + ".zip", "r") as z:
        z.extractall(".")

In [ ]:
# 資料預處理
import pandas as pd

# 從train資料夾取得檔名、放入filenames
filenames = os.listdir("./train")
# store the label for each image file
categories = []

# 使用訓練資料檔名 dog.x.jpg、cat.x.jpg來生成1與0的標籤
for filename in filenames:
    # 分割檔名，僅擷取前頭的元素(dog/cat)、
    # 以dog為 1、cat為0設定好標籤後放入category
    category = filename.split('.')[0]
    if category == 'dog':
        # 若為dog則是標籤1
        categories.append(1)
    else:
        # 若為cat則是標籤0 
        categories.append(0)

# 在df行 filename放入檔名filenames
# 在行category放入標籤數值categories
df = pd.DataFrame({'filename': filenames,
                   'category': categories})
df.head()
# 將資料框架前五行輸出
df.head()


In [ ]:
df['category'].value_counts().plot.bar()

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
import random
%matplotlib inline

# 隨機抽取16張 
sample = random.sample(filenames, 16) 

#繪圖區域尺寸為12 × 12 
plt.figure(figsize=(12, 12)) 

for i in range(0, 16): 
    # 從4 × 4格線的左上角開始依序繪製 
    plt.subplot(4, 4, i+1) 
    #放入 sample的第i 項的圖像 
    fname = sample[i] 
    # 從檔案夾train中讀取圖像資料
    image = load_img("./train/"+fname)
    # 描繪圖像
    plt.imshow(image)
    plt.axis('off') # 隱藏刻度
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split

# 90%做為訓練資料，10%做為驗證資料
train_df, validate_df = train_test_split(df, test_size=0.1) 
# 重新配置列的索引
train_df = train_df.reset_index()
validate_df = validate_df.reset_index()

# 取得訓練資料的數量 
total_train = train_df.shape[0] 
# 取得驗證資料的數量
total_validate = validate_df.shape[0]

# 輸出訓練資料、驗證資料的數量
print(total_train)
print(total_validate)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 重新調整圖像尺寸
img_width, img_height = 224, 224
target_size = (img_width, img_height) 
# 批次大小
batch_size = 16 
# 檔名的欄位名稱，標籤的欄位名稱
x_col, y_col = 'filename', 'category' 
# 設定flow_from_dataframe()的class_mode數值 
# 此範例為二元分類，設定值為'binary'
class_mode = 'binary'

# 生成Generator用於加工圖像
train_datagen = ImageDataGenerator(rescale=1./255,        # 正規化
                                   rotation_range=15,     # 隨機旋轉
                                   shear_range=0.2,       # 錯切
                                   zoom_range=0.2,        # 放大
                                   horizontal_flip=True,  # 水平翻轉
                                   width_shift_range=0.1, # 左右移動
                                   height_shift_range=0.1 # 上下移動
                                  )

# 當flow_from_dataframe()引數為class_mode = "binary"時，
# 標籤(train_df中的y_col = 'category')該行須為文字格式
# 因此需要將將1與0的數字轉換為文字
train_df['category'] = train_df['category'].astype(str)

# 使用Generator生成加工完的圖像
train_generator = train_datagen.flow_from_dataframe(train_df,    
                                                    # 圖像資料目錄
                                                    "./train/",  
                                                    # 放置檔名的欄位名稱
                                                    x_col=x_col, 
                                                    # 放置標籤的欄位名稱
                                                    y_col=y_col, 
                                                    # 二元分類
                                                    class_mode=class_mode,  
                                                    # 圖像尺寸
                                                    target_size=target_size,
                                                    # 批次大小
                                                    batch_size=batch_size)


In [ ]:
# 建立預處理圖像的Generator 
# 無需進行資料擴增，所以只進行數值變換即可
valid_datagen = ImageDataGenerator(rescale=1./255)
# 當flow_from_dataframe()引數為class_mode = "binary"時，
# 標籤(train_df中的y_col = 'category')那一欄需為文字格式
# 因此需要將1與0的數字轉換為文字
validate_df['category'] = validate_df['category'].astype(str)

# 使用Generator產生預處理完的圖像
valid_generator = valid_datagen.flow_from_dataframe(validate_df, 
                                                    # 圖像資料目錄
                                                    "./train/", 
                                                    # 放置檔名的欄位名稱
                                                    x_col=x_col, 
                                                    # 放置標籤的欄位名稱
                                                    y_col=y_col, 
                                                    # 二元分類
                                                    class_mode=class_mode,  
                                                    # 圖像尺寸
                                                    target_size=target_size, 
                                                    # 批次大小
                                                    batch_size=batch_size)


In [ ]:
# 從訓練資料取出1個樣本、使用reset_index()重新配置索引
# 用drop=True來刪除原本的索引
example_df = train_df.sample(n=1).reset_index(drop=True)
# 建立DataFrameIterator物件
example_generator = train_datagen.flow_from_dataframe(example_df,
                                                      "./train/",
                                                      x_col='filename',
                                                      y_col='category',
                                                      target_size=target_size)
plt.figure(figsize=(12, 12))
# 顯示預處理後的9種樣貌
for i in range(0, 9):
    # 在 3 × 3 的框架中由左上角開始依序繪圖
    plt.subplot(3, 3, i+1)
    for X_batch, Y_batch in example_generator:  
        # 取出X_batch的第1個圖像資料
        image = X_batch[0]
        # 繪製完取出的圖像後就break
        plt.imshow(image)
        break
plt.show()


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers

# 建立Sequentual
model = Sequential()

# 輸入資料之形狀
input_shape = (img_width, img_height, 3)

# 第1層:第一層卷積層
model.add(Conv2D(filters=32,               # 卷積核數量為 32
                 kernel_size=(3, 3),       # 使用 3 × 3卷積核
                 padding='same',           # 執行墊零
                 activation='relu',        # 激活函數為 ReLU
                 input_shape=input_shape)) # 輸入資料之形狀

# 第2層:池化層
model.add(MaxPooling2D(pool_size=(2, 2)))
# 丟棄率25% 
model.add(Dropout(0.25)) 

# 第3層:第二層卷積層
model.add(Conv2D(filters = 64,        # 卷積核數量為64
                 kernel_size = (3,3), # 使用3 × 3卷積核
                 padding='same',      # 執行墊零 
                 activation='relu'))  # 激活函數 ReLU

# 第4層:池化層
model.add(MaxPooling2D(pool_size=(2, 2)))
# 丟棄率 25%
model.add(Dropout(0.25))

# 第5層:第三層卷積層
model.add(Conv2D(filters=128,        # 卷積核數量為128
                 kernel_size=(3, 3), # 使用3 × 3卷積核
                 padding='same',     # 執行墊零
                 activation='relu')) # 激活函數為 ReLU

# 第6層:池化層
model.add(MaxPooling2D(pool_size=(2, 2)))
# 丟棄率 25%
model.add(Dropout(0.25))

# 將四維矩陣(batch_size, rows, cols, channels)拉直為二維矩陣(batch_size, channels) 
model.add(GlobalMaxPooling2D())

# 第7層
model.add(Dense(128,                # 神經元數量
                activation='relu')) # 激活函數為 ReLU
# 丟棄率 25% 
model.add(Dropout(0.25)) 

# 第8層:輸出層 
model.add(Dense(1,                     # 神經元數量為1個 
                activation='sigmoid')) # 激活函數為Sigmoid  
# 編譯模型 
model.compile(loss='binary_crossentropy',    # 二進位交叉熵誤差 
              metrics=['accuracy'],          # 將準確率指定為評價指標
              optimizer=optimizers.RMSprop() # 使用RMSprop優化器
)


In [ ]:
import math
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, Callback

# 對學習率進行排程
def step_decay(epoch):
    initial_lrate = 0.001 # 初始學習率
    drop = 0.5            # 衰減率為50%
    epochs_drop = 10.0    # 每10次訓練週期進行衰減
    lrate = initial_lrate * math.pow(drop,
                                     math.floor((epoch)/epochs_drop))
    return lrate

# 建立學習率回呼
lrate = LearningRateScheduler(step_decay)

# 監控學習進度、回呼提前中止檢查
earstop = EarlyStopping(monitor='val_loss', # 監控損失狀況
                        min_delta=0,        # 用於判定為有改善的最小變化值
                        patience=5)         # 將用於判定為沒改善的訓練週期次數加大到5

epochs = 40 # 訓練週期

history = model.fit(train_generator,                            # 訓練資料
                    epochs=epochs,                              # 訓練週期
                    steps_per_epoch=total_train//batch_size,    # 訓練時的步驟數
                    validation_data=valid_generator,            # 驗證資料
                    validation_steps=total_validate//batch_size,# 驗證時的步驟數
                    verbose=1,                                  # 輸出訓練進度
                    callbacks=[lrate, earstop])                 # 回呼
